In [41]:
import plotly.plotly as py
from plotly.grid_objs import Grid, Column
from plotly.tools import FigureFactory as FF 

import pandas as pd
import time
url = 'bubbledata.csv'
dataset = pd.read_csv(url)
table = FF.create_table(dataset.head(10))
py.iplot(table, filename='animations-gapminder-data-preview')

D:\anaconda\lib\site-packages\plotly\tools.py:1545: UserWarning:

plotly.tools.FigureFactory.create_table is deprecated. Use plotly.figure_factory.create_table

D:\anaconda\lib\site-packages\IPython\core\display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



In [42]:
months_from_col = set(dataset['month'])
months_ints = sorted(list(months_from_col))
months = [str(month) for month in months_ints]
areas = []
for area in dataset['area']:
    if area not in areas: 
        areas.append(area)
columns = []
for month in months:
    for area in areas:
        dataset_by_month = dataset[dataset['month'] == int(month)]
        dataset_by_month_and_cont = dataset_by_month[dataset_by_month['area'] == area]
        for col_name in dataset_by_month_and_cont:
            column_name = '{month}_{area}_{header}_grid'.format(
                month=month, area=area, header=col_name
            )
            a_column = Column(list(dataset_by_month_and_cont[col_name]), column_name)
            columns.append(a_column)

In [43]:
grid = Grid(columns)
url = py.grid_ops.upload(grid, 'grid'+str(time.time()), auto_open=False)
figure = {
    'data': [],
    'layout': {},
    'frames': [],
    'config': {'scrollzoom': True}
}

# fill in most of layout
figure['layout']['xaxis'] = {'range': [40,200], 'title': 'Giraffe activation', 'gridcolor': '#FFFFFF'}
figure['layout']['yaxis'] = {'title': 'Scale of migration', 'type': 'log', 'gridcolor': '#FFFFFF'}
figure['layout']['hovermode'] = 'closest'
figure['layout']['plot_bgcolor'] = 'rgb(223, 232, 243)'
sliders_dict = {
    'active': 0,
    'yanchor': 'top',
    'xanchor': 'left',
    'currentvalue': {
        'font': {'size': 20},
        'prefix': 'Month:',
        'visible': True,
        'xanchor': 'right'
    },
    'transition': {'duration': 300, 'easing': 'cubic-in-out'},
    'pad': {'b': 10, 't': 50},
    'len': 0.9,
    'x': 0.1,
    'y': 0,
    'steps': []
}
figure['layout']['updatemenus'] = [
    {
        'buttons': [
            {
                'args': [None, {'frame': {'duration': 500, 'redraw': False},
                         'fromcurrent': True, 'transition': {'duration': 300, 'easing': 'quadratic-in-out'}}],
                'label': 'Play',
                'method': 'animate'
            },
            {
                'args': [[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate',
                'transition': {'duration': 0}}],
                'label': 'Pause',
                'method': 'animate'
            }
        ],
        'direction': 'left',
        'pad': {'r': 10, 't': 87},
        'showactive': False,
        'type': 'buttons',
        'x': 0.1,
        'xanchor': 'right',
        'y': 0,
        'yanchor': 'top'
    }
]

custom_colors = {
    'a': 'rgb(171, 99, 250)',
    'b': 'rgb(230, 99, 250)',
    'c': 'rgb(99, 110, 250)',
    'd': 'rgb(25, 211, 243)',
    'e': 'rgb(50, 170, 255)',
    'f': 'rgb(181,160,48)'
}

In [44]:
col_name_template = '{month}_{area}_{header}_grid'
month=1
for area in areas:
    data_dict = {
        'xsrc': grid.get_column_reference(col_name_template.format(
            month=month, area=area, header='giraffe'
        )),
        'ysrc': grid.get_column_reference(col_name_template.format(
            month=month, area=area, header='number'
        )),
        'mode': 'markers',
        'textsrc': grid.get_column_reference(col_name_template.format(
            month=month, area=area, header='area'
        )),
        'marker': {
            'sizemode': 'area',
            'sizeref': 1,
            'sizesrc': grid.get_column_reference(col_name_template.format(
                 month=month, area=area, header='number'
            )),
            'color': custom_colors[area]
        },
        'name': area
    }
    figure['data'].append(data_dict)

In [45]:
for month in months:
    frame = {'data': [], 'name': str(month)}
    for area in areas:
        data_dict = {
            'xsrc': grid.get_column_reference(col_name_template.format(
                month=month, area=area, header='giraffe'
            )),
            'ysrc': grid.get_column_reference(col_name_template.format(
                month=month, area=area, header='number'
            )),
            'mode': 'markers',
            'textsrc': grid.get_column_reference(col_name_template.format(
                month=month, area=area, header='area'
                )),
            'marker': {
                'sizemode': 'area',
                'sizeref': 1,
                'sizesrc': grid.get_column_reference(col_name_template.format(
                    month=month, area=area, header='number'
                )),
                'color': custom_colors[area]
            },
            'name': area
        }
        frame['data'].append(data_dict)

    figure['frames'].append(frame)
    slider_step = {'args': [
        [month],
        {'frame': {'duration': 300, 'redraw': False},
         'mode': 'immediate',
       'transition': {'duration': 300}}
     ],
     'label': month,
     'method': 'animate'}
    sliders_dict['steps'].append(slider_step)

figure['layout']['sliders'] = [sliders_dict]

In [46]:
py.icreate_animations(figure, 'Giraffe number floating tendency'+str(time.time()))